In [1]:
import os
import re

import pandas as pd
import numpy as np

import plotly.express as px
import networkx as nx

import json
import copy

In [2]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [23]:
from networks.RulesGraphManager import RulesGraphManager as RGM
from grouper.NxGrouper import NxGrouper

from echarts.EgraphForce import EgraphForce
from echarts.EgraphStandard import EgraphStandard
from echarts.JupyterEcharts import JupyterEcharts

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

### Data Preparation

#### Data Cleaning

In [4]:
def get_cleaned_df():
    global COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID, COL_VALUE, COL_QUANTITY
    COL_CUSTOMER_ID = 'Customer ID'
    COL_ORDER_DATE = 'InvoiceDate'
    COL_ORDER_ID = 'Invoice'
    COL_VALUE = 'Sales'
    COL_QUANTITY = 'Quantity'
    
    pathname = os.path.join("F:\\Data\\datas", "online_retail_II.csv")
    df = pd.read_csv(
        pathname, 
        dtype = {'Customer ID': str, 'Invoice': str},
        parse_dates = ['InvoiceDate']
    )
    
    df = df.dropna()
    df['Sales'] = df['Price'] * df['Quantity']
    
    price_0 = df[df['Price'] == 0].index
    df = df.drop(index=price_0)
    
    # From this data below, we know that the canceled invoice (the initialize invoice) already dropped out in our dataset
    # Our action is to get rid of the canceled Invoice. The canceled invoice code starts with "C"
#     display(df[(df[COL_CUSTOMER_ID] == '16321.0') & (df['StockCode'] == '22087')])
    df = df[~df['Invoice'].str.startswith('C')]
    df['Description'] = df['Description'].map(lambda x: x.strip())
    df['Description'] = df['Description'].map(lambda x: re.sub(r'\s{2,}', ' ', x))
    
    return df[['Invoice', 'Description']]

In [5]:
df = get_cleaned_df()

#### Data Encoding

In [6]:
def encoding_data(df, COL_ITEM_ID, COL_ORDER_ID):
    df = df.copy()
    df[COL_ITEM_ID] = df[COL_ITEM_ID].transform(lambda x: [x])
    df = df.groupby(COL_ORDER_ID, as_index=False).sum()[COL_ITEM_ID]
    df = df.map(lambda x: list(set(x)))
                          
    encoder = TransactionEncoder()
    one_hot_transactions = pd.DataFrame(
        encoder.fit(df).transform(df), columns=encoder.columns_
    )
    
    return one_hot_transactions

In [7]:
df_transactions = encoding_data(df, 'Description', 'Invoice')

### Data Mining (Rules Extraction)

In [8]:
frequent_itemsets = apriori(df_transactions, min_support= 0.01, use_colnames=True, max_len = 3, low_memory=True)
rules = association_rules(frequent_itemsets, metric="support", min_threshold = 0.02)

In [9]:
rules['antecedents'] = rules['antecedents'].map(lambda x: ''.join(list(x)))
rules['consequents'] = rules['consequents'].map(lambda x: ''.join(list(x)))

In [10]:
rules.shape

(22, 10)

### Network Visualization with Echarts

In [14]:
myRGM = RGM(rules, 'antecedents', 'consequents')
df_nodes, df_edges = myRGM.get_graph_features()

In [27]:
df_nodes = NxGrouper.greedy_modularity_communities(df_nodes, df_edges, 2)

In [30]:
df_nodes

,nodes,adjacent_edges,out_edges,in_edges,mutual_edges,support,category
0,LUNCH BAG SUKI DESIGN,3,3,3,3,0.053342,LUNCH BAG SUKI DESIGN
1,LUNCH BAG BLACK SKULL.,3,3,3,3,0.054018,LUNCH BAG BLACK SKULL.
2,LUNCH BAG CARS BLUE,2,2,2,2,0.048608,LUNCH BAG BLACK SKULL.
3,LUNCH BAG SPACEBOY DESIGN,2,2,2,2,0.050691,LUNCH BAG SUKI DESIGN
4,GREEN REGENCY TEACUP AND SAUCER,1,1,1,1,0.025237,GREEN REGENCY TEACUP AND SAUCER
5,HEART OF WICKER LARGE,1,1,1,1,0.047148,HEART OF WICKER LARGE
6,HOME BUILDING BLOCK WORD,1,1,1,1,0.049528,HOME BUILDING BLOCK WORD
7,RED HANGING HEART T-LIGHT HOLDER,1,1,1,1,0.044118,RED HANGING HEART T-LIGHT HOLDER
8,SWEETHEART CERAMIC TRINKET BOX,1,1,1,1,0.031675,SWEETHEART CERAMIC TRINKET BOX
9,WOODEN FRAME ANTIQUE WHITE,1,1,1,1,0.047959,WOODEN FRAME ANTIQUE WHITE


In [28]:
force = EgraphForce(
    df_edges, 
    df_nodes, 
    col_source='antecedents', 
    col_target='consequents', 
    col_name='nodes',
)
option = force.get_option()

In [29]:
JupyterEcharts.show(option)